In [21]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import time
import math
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from preprocess import prepare_df
from sklearn.metrics import r2_score



In [5]:
data = pd.read_csv('../data/df_final.csv', index_col=0).drop(columns=['lat', 'lon', 'elv','date','c4','whc'])
good_sites = pd.read_csv("../data/df_20210507.csv", low_memory=False )['sitename'].unique()

In [6]:
df_sensor, df_meta, df_gpp = prepare_df(data,sites=good_sites)

In [64]:
sites_to_train = list(range(len(df_sensor)))
sites_to_train.remove(0)
sites_to_test = [0]

x_train = [pd.concat((df_sensor[i], df_meta[i]), axis=1).values for i in sites_to_train]
conditional_train = [df_meta[i].values for i in sites_to_train]
y_train = [df_gpp[i].values.reshape(-1,1) for i in sites_to_train]

x_test = [pd.concat((df_sensor[i],df_meta[i]), axis=1).values for i in sites_to_test]
conditional_test = [df_meta[i].values for i in sites_to_test]
y_test = [df_gpp[i].values.reshape(-1,1) for i in sites_to_test]

In [65]:
class Encoder(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.input_features = input_features
        self.output_features = output_features

        self.rnn = nn.LSTM(input_size=input_features, hidden_size=output_features)
    
    def forward(self, x):
        outputs, (h, c) = self.rnn(x)

        return outputs.squeeze(1) #shape=(seq_len, num_dir * output_features)

class Reparametrize(nn.Module):
    def __init__(self, encoder_output, latent_size):
        super().__init__()
        self.encoder_output = encoder_output
        self.latent_size = latent_size

        self.fc_to_mean = nn.Linear(encoder_output, latent_size)
        self.fc_to_logvar = nn.Linear(encoder_output, latent_size)

        self.fc_to_r_mean = nn.Linear(encoder_output, 1)
        self.fc_to_r_logvar = nn.Linear(encoder_output, 1)

        self.fc_r_to_pzmean = nn.Linear(1, latent_size)

        nn.init.xavier_uniform_(self.fc_to_mean.weight)
        nn.init.xavier_uniform_(self.fc_to_logvar.weight)
        nn.init.xavier_uniform_(self.fc_to_r_mean.weight)
        nn.init.xavier_uniform_(self.fc_to_r_logvar.weight)

    def forward(self, x):
        self.mean = self.fc_to_mean(x)
        self.logvar = self.fc_to_logvar(x)

        self.r_mean = self.fc_to_r_mean(x)
        self.r_logvar = self.fc_to_r_logvar(x)

        std = torch.exp(0.5 * self.logvar)
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(self.mean)

        std = torch.exp(0.5 * self.r_logvar)
        eps = torch.randn_like(std)
        r = eps.mul(std).add_(self.r_mean)

        return z, self.mean, self.logvar, self.fc_r_to_pzmean(r), self.r_mean, self.r_logvar

class Decoder(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        self.input_features = input_features
        self.output_features = output_features

        self.rnn = nn.LSTM(input_size=input_features, hidden_size=output_features)

        self.rnn_to_output = nn.Sequential(
            nn.Linear(self.output_features, self.output_features),
            nn.Tanh(),
            nn.Linear(self.output_features, self.output_features),
        )

    def forward(self, x):
        outputs, (h, c) = self.rnn(x)
        outputs = outputs.squeeze(1)
        return self.rnn_to_output(outputs) #shape=(seq_len, num_dir * output_features)

class Model(nn.Module):
    def __init__(self, encoder, decoder, reparam):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.reparam = reparam

    def forward(self, x, conditional):
        x = self.encoder(x) # x has the conditional
        z, mean, logvar, pz_mean, r_mean, r_logvar = self.reparam(x)

        # concat the conditoinal to z
        z1 = torch.cat([z, conditional], dim=1)
        
        # decode
        x = self.decoder(z1.unsqueeze(1))

        return x, mean, logvar, pz_mean, r_mean, r_logvar

In [66]:
def loss_fn(x_decoded, x, mu, logvar, w, pz_mean, r_mean, r_logvar, r_input):
    kl_loss = w * (-0.5) * torch.mean(1 + logvar - (mu - pz_mean).pow(2) - logvar.exp())
    recon_loss = F.mse_loss(x_decoded, x)
    label_loss = torch.mean(0.5 * (r_mean - r_input).pow(2) / r_logvar.exp() + 0.5 * r_logvar)
    return kl_loss + recon_loss + label_loss, kl_loss, recon_loss, label_loss

In [75]:
ENCODER_INPUT_DIM = x_train[0].shape[1]
ENCODER_OUTPUT_DIM = 16
REPARAM_INPUT_DIM = ENCODER_OUTPUT_DIM
LATENT_DIM = 64
REPARAM_OUTPUT_DIM = LATENT_DIM
DECODER_INPUT_DIM = LATENT_DIM + conditional_train[0].shape[1]
DECODER_OUTPUT_DIM = len(df_sensor[0].columns)

DEVICE = 'cuda:7'

encoder = Encoder(ENCODER_INPUT_DIM, ENCODER_OUTPUT_DIM).to(DEVICE)
decoder = Decoder(DECODER_INPUT_DIM, DECODER_OUTPUT_DIM).to(DEVICE)
reparam = Reparametrize(REPARAM_INPUT_DIM, REPARAM_OUTPUT_DIM).to(DEVICE)
model = Model(encoder, decoder, reparam).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [76]:
EPOCHS = 200
for epoch in range(EPOCHS):
    train_loss = 0.0
    train_kl_loss = 0.0
    train_recon_loss = 0.0
    train_label_loss = 0.0
    train_r2 = 0.0

    test_loss = 0.0
    test_kl_loss = 0.0
    test_recon_loss = 0.0
    test_label_loss = 0.0
    test_r2 = 0.0

    model.train()
    for (x, c, y) in zip(x_train, conditional_train, y_train):
        # Convert to tensors
        x = torch.FloatTensor(x).unsqueeze(1).to(DEVICE)
        c = torch.FloatTensor(c).to(DEVICE)
        y = torch.FloatTensor(y).to(DEVICE)

        # Get predictions
        out, mean, logvar, pz_mean, r_mean, r_logvar = model(x, c)

        # Remove the conditional from x
        x = x.squeeze(1)[:, :len(df_sensor[0].columns)]

        # Get loss and update
        optimizer.zero_grad()
        loss, kl_loss, recon_loss, label_loss = loss_fn(out, x, mean, logvar, 1, pz_mean, r_mean, r_logvar, y)
        loss.backward()
        optimizer.step()
    
        # Update losses
        train_loss += loss.item()
        train_kl_loss += kl_loss.item()
        train_recon_loss += recon_loss.item()
        train_label_loss += label_loss.item()
        train_r2 += r2_score(y.detach().cpu().numpy(), r_mean.detach().cpu().numpy())

    model.eval()
    with torch.no_grad():
        for (x, c, y) in zip(x_test, conditional_test, y_test):
            # Convert to tensors
            x = torch.FloatTensor(x).unsqueeze(1).to(DEVICE)
            c = torch.FloatTensor(c).to(DEVICE)
            y = torch.FloatTensor(y).to(DEVICE)

            # Get predictions
            out, mean, logvar, pz_mean, r_mean, r_logvar = model(x, c)

            # Remove the conditional from x
            x = x.squeeze(1)[:, :len(df_sensor[0].columns)]

            # Get loss
            loss, kl_loss, recon_loss, label_loss = loss_fn(out, x, mean, logvar, 1, pz_mean, r_mean, r_logvar, y)
        
            # Update losses
            test_loss += loss.item()
            test_kl_loss += kl_loss.item()
            test_recon_loss += recon_loss.item()
            test_label_loss += label_loss.item()
            test_r2 += r2_score(y.detach().cpu().numpy(), r_mean.detach().cpu().numpy())

    

    train_loss /= len(x_train)
    train_kl_loss /= len(x_train)
    train_recon_loss /= len(x_train)
    train_label_loss /= len(x_train)
    train_r2 /= len(x_train)

    test_loss /= len(x_test)
    test_kl_loss /= len(x_test)
    test_recon_loss /= len(x_test)
    test_label_loss /= len(x_test)
    test_r2 /= len(x_test)
    
    print(f"Epoch: {epoch+1}/{EPOCHS}")
    print(f"Train loss: {train_loss:.4f} | Recon loss: {train_recon_loss:.4f} | KL loss: {train_kl_loss:.4f} | Label loss: {train_label_loss:.4f}")
    print(f"Test loss: {test_loss:.4f} | Recon loss: {test_recon_loss:.4f} | KL loss: {test_kl_loss:.4f} | Label loss: {test_label_loss:.4f}")
    print(f"Train R2: {train_r2:.4f}")
    print(f"Test R2: {test_r2:.4f}")

Epoch: 1/200
Train loss: 2.4540 | Recon loss: 1.0315 | KL loss: 0.3544 | Label loss: 1.0681
Test loss: 2.1024 | Recon loss: 1.0184 | KL loss: 0.3212 | Label loss: 0.7628
Train R2: -0.8131
Test R2: -0.3210
Epoch: 2/200
Train loss: 1.9663 | Recon loss: 1.0106 | KL loss: 0.2981 | Label loss: 0.6576
Test loss: 1.8048 | Recon loss: 1.0025 | KL loss: 0.2705 | Label loss: 0.5318
Train R2: -0.2628
Test R2: -0.0017
Epoch: 3/200
Train loss: 1.6544 | Recon loss: 0.9836 | KL loss: 0.2601 | Label loss: 0.4107
Test loss: 1.5882 | Recon loss: 0.9600 | KL loss: 0.2482 | Label loss: 0.3799
Train R2: 0.0986
Test R2: 0.2030
Epoch: 4/200
Train loss: 1.3694 | Recon loss: 0.9048 | KL loss: 0.2340 | Label loss: 0.2305
Test loss: 1.3785 | Recon loss: 0.8808 | KL loss: 0.2241 | Label loss: 0.2735
Train R2: 0.3391
Test R2: 0.3283
Epoch: 5/200
Train loss: 1.0769 | Recon loss: 0.7716 | KL loss: 0.2150 | Label loss: 0.0903
Test loss: 1.2071 | Recon loss: 0.7960 | KL loss: 0.2022 | Label loss: 0.2089
Train R2: 0.48

Epoch: 41/200
Train loss: -0.2127 | Recon loss: 0.5582 | KL loss: 0.0253 | Label loss: -0.7962
Test loss: 1.4942 | Recon loss: 0.6340 | KL loss: 0.0381 | Label loss: 0.8221
Train R2: 0.8874
Test R2: 0.7050
Epoch: 42/200
Train loss: -0.2205 | Recon loss: 0.5549 | KL loss: 0.0254 | Label loss: -0.8007
Test loss: 1.5392 | Recon loss: 0.6337 | KL loss: 0.0373 | Label loss: 0.8682
Train R2: 0.8879
Test R2: 0.7055
Epoch: 43/200
Train loss: -0.2275 | Recon loss: 0.5521 | KL loss: 0.0256 | Label loss: -0.8051
Test loss: 1.5778 | Recon loss: 0.6280 | KL loss: 0.0372 | Label loss: 0.9125
Train R2: 0.8884
Test R2: 0.7062
Epoch: 44/200
Train loss: -0.2350 | Recon loss: 0.5485 | KL loss: 0.0258 | Label loss: -0.8093
Test loss: 1.6203 | Recon loss: 0.6238 | KL loss: 0.0366 | Label loss: 0.9600
Train R2: 0.8889
Test R2: 0.7068
Epoch: 45/200
Train loss: -0.2416 | Recon loss: 0.5457 | KL loss: 0.0261 | Label loss: -0.8134
Test loss: 1.6761 | Recon loss: 0.6306 | KL loss: 0.0375 | Label loss: 1.0081
Tra

KeyboardInterrupt: 

In [70]:
x1 = torch.FloatTensor(x_train[0]).unsqueeze(1).to(DEVICE)
c1 = torch.FloatTensor(conditional_train[0]).to(DEVICE)

x2 = torch.FloatTensor(x_train[20]).unsqueeze(1).to(DEVICE)
c2 = torch.FloatTensor(conditional_train[20]).to(DEVICE)



out1, mean1, logvar1, _, _, _ = model(x1, c1)
out2, mean2, logvar2, _, _ , _  = model(x2, c2)



torch.Size([6210, 8])

In [71]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
mean = pca.fit_transform(torch.cat([mean1,mean2]).detach().cpu())

In [72]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=mean[:1095,0], y=mean[:1095,1],
                    mode='markers',
                    name='Site 0', 
                    marker=dict(size=3)))
fig.add_trace(go.Scatter(x=mean[1095:,0], y=mean[1095:,1],
                    mode='markers',
                    name='Site 20', 
                    marker=dict(size=3)))             